# Training at scale with AI Platform Training Service
**Learning Objectives:**
  1. Learn how to organize your training code into a Python package
  1. Train your model using cloud infrastructure via Google Cloud AI Platform Training Service
  1. (optional) Learn how to run your training package using Docker containers and push training Docker images on a Docker registry

## Introduction

In this notebook we'll make the jump from training locally, to do training in the cloud. We'll take advantage of Google Cloud's [AI Platform Training Service](https://cloud.google.com/ai-platform/). 

AI Platform Training Service is a managed service that allows the training and deployment of ML models without having to provision or maintain servers. The infrastructure is handled seamlessly by the managed service for us.

Each learning objective will correspond to a __#TODO__ in the [student lab notebook](../labs/1_training_at_scale.ipynb) -- try to complete that notebook first before reviewing this solution notebook.

In [1]:
from google import api_core
from google.cloud import bigquery

Change your project name and bucket name in the cell below if necessary.

In [2]:
# Change with your own bucket and project below:
PROJECT = !gcloud config get-value project
PROJECT = PROJECT[0]
BUCKET = PROJECT
REGION = "us-central1"

OUTDIR = "gs://{bucket}/taxifare/data".format(bucket=BUCKET)

%env PROJECT=$PROJECT
%env BUCKET=$BUCKET
%env REGION=$REGION
%env OUTDIR=$OUTDIR
%env TFVERSION=2.3

env: BUCKET=dsparing-sandbox
env: OUTDIR=gs://dsparing-sandbox/taxifare/data
env: PROJECT=dsparing-sandbox
env: REGION=us-central1
env: TFVERSION=2.3


Confirm below that the bucket is regional and its region equals to the specified region:

In [3]:
%%bash
gsutil ls -Lb gs://$BUCKET | grep "gs://\|Location"
echo $REGION

gs://dsparing-sandbox/ :
	Location type:			region
	Location constraint:		US-CENTRAL1
us-central1


In [4]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION
gcloud config set ai_platform/region $REGION

Updated property [core/project].
Updated property [compute/region].
Updated property [ai_platform/region].


## Create BigQuery tables

If you have not already created a BigQuery dataset for our data, run the following cell:

In [5]:
bq = bigquery.Client(project=PROJECT)
dataset = bigquery.Dataset(bq.dataset("taxifare"))

try:
    bq.create_dataset(dataset)
    print("Dataset created")
except api_core.exceptions.Conflict:
    print("Dataset already exists")

Dataset already exists


Let's create a table with 1 million examples.

Note that the order of columns is exactly what was in our CSV files.

In [6]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_training_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `nyc-tlc.yellow.trips`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 1000)) = 1
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 894.88query/s]                         


""


Make the validation dataset be 1/10 the size of the training dataset.

In [7]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_valid_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `nyc-tlc.yellow.trips`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 10000)) = 2
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 981.28query/s]                         


""


## Export the tables as CSV files

In [8]:
%%bash

echo "Deleting current contents of $OUTDIR"
gsutil -m -q rm -rf $OUTDIR

echo "Extracting training data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_training_data \
   $OUTDIR/taxi-train-*.csv

echo "Extracting validation data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_valid_data \
   $OUTDIR/taxi-valid-*.csv

gsutil ls -l $OUTDIR

Deleting current contents of gs://dsparing-sandbox/taxifare/data
Extracting training data to gs://dsparing-sandbox/taxifare/data
Extracting validation data to gs://dsparing-sandbox/taxifare/data
  88345235  2021-08-25T21:35:11Z  gs://dsparing-sandbox/taxifare/data/taxi-train-000000000000.csv
   8725746  2021-08-25T21:35:22Z  gs://dsparing-sandbox/taxifare/data/taxi-valid-000000000000.csv
TOTAL: 2 objects, 97070981 bytes (92.57 MiB)


Waiting on bqjob_r3a412c726d11906a_0000017b7f3cca7e_1 ... (22s) Current status: DONE   
Waiting on bqjob_r8266a1f8a81f9ca_0000017b7f3d2d22_1 ... (1s) Current status: DONE   


In [9]:
!gsutil cat gs://$BUCKET/taxifare/data/taxi-train-000000000000.csv | head -2

56.5,2012-11-24 18:06:45 UTC,-73.77674,40.645477,-73.993984,40.69777,2,unused
20.83,2013-05-30 22:25:59 UTC,-74.001792,40.739539,-73.956885,40.746525,2,unused


## Make code compatible with AI Platform Training Service
In order to make our code compatible with AI Platform Training Service we need to make the following changes:

1. Upload data to Google Cloud Storage 
2. Move code into a trainer Python package
4. Submit training job with `gcloud` to train on AI Platform

### Upload data to Google Cloud Storage (GCS)

Cloud services don't have access to our local files, so we need to upload them to a location the Cloud servers can read from. In this case we'll use GCS.

To do this run the notebook [0_export_data_from_bq_to_gcs.ipynb](../0_export_data_from_bq_to_gcs.ipynb), which will export the taxifare data from BigQuery directly into a GCS bucket. If all ran smoothly, you should be able to list the data bucket by running the following command:


In [10]:
!gsutil ls gs://$BUCKET/taxifare/data

gs://dsparing-sandbox/taxifare/data/taxi-train-000000000000.csv
gs://dsparing-sandbox/taxifare/data/taxi-valid-000000000000.csv


### Move code into a Python package


The first thing to do is to convert your training code snippets into a regular Python package.

A Python package is simply a collection of one or more `.py` files along with an `__init__.py` file to identify the containing directory as a package. The `__init__.py` sometimes contains initialization code but for our purposes an empty file suffices.

#### Create the package directory

Our package directory contains 3 files:

In [11]:
ls ./taxifare/trainer/

__init__.py  __pycache__/  model.py  task.py


#### Paste existing code into model.py

A Python package requires our code to be in a .py file, as opposed to notebook cells. So, we simply copy and paste our existing code for the previous notebook into a single file.

In the cell below, we write the contents of the cell into `model.py` packaging the model we 
developed in the previous labs so that we can deploy it to AI Platform Training Service.  

In [12]:
%%writefile ./taxifare/trainer/model.py
import datetime
import logging
import os

import numpy as np
import tensorflow as tf
from tensorflow import feature_column as fc
from tensorflow.keras import activations, callbacks, layers, models

logging.info(tf.version.VERSION)


CSV_COLUMNS = [
    "fare_amount",
    "pickup_datetime",
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
    "passenger_count",
    "key",
]
LABEL_COLUMN = "fare_amount"
DEFAULTS = [[0.0], ["na"], [0.0], [0.0], [0.0], [0.0], [0.0], ["na"]]
DAYS = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]


def features_and_labels(row_data):
    for unwanted_col in ["key"]:
        row_data.pop(unwanted_col)
    label = row_data.pop(LABEL_COLUMN)
    return row_data, label


def load_dataset(pattern, batch_size, num_repeat):
    dataset = tf.data.experimental.make_csv_dataset(
        file_pattern=pattern,
        batch_size=batch_size,
        column_names=CSV_COLUMNS,
        column_defaults=DEFAULTS,
        num_epochs=num_repeat,
    )
    return dataset.map(features_and_labels)


def create_train_dataset(pattern, batch_size):
    dataset = load_dataset(pattern, batch_size, num_repeat=None)
    return dataset.prefetch(1)


def create_eval_dataset(pattern, batch_size):
    dataset = load_dataset(pattern, batch_size, num_repeat=1)
    return dataset.prefetch(1)


def parse_datetime(s):
    if type(s) is not str:
        s = s.numpy().decode("utf-8")
    return datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S %Z")


def euclidean(params):
    lon1, lat1, lon2, lat2 = params
    londiff = lon2 - lon1
    latdiff = lat2 - lat1
    return tf.sqrt(londiff * londiff + latdiff * latdiff)


def get_dayofweek(s):
    ts = parse_datetime(s)
    return DAYS[ts.weekday()]


@tf.function
def dayofweek(ts_in):
    return tf.map_fn(
        lambda s: tf.py_function(get_dayofweek, inp=[s], Tout=tf.string), ts_in
    )


@tf.function
def fare_thresh(x):
    return 60 * activations.relu(x)


def transform(inputs, NUMERIC_COLS, STRING_COLS, nbuckets):
    # Pass-through columns
    transformed = inputs.copy()
    del transformed["pickup_datetime"]

    feature_columns = {
        colname: fc.numeric_column(colname) for colname in NUMERIC_COLS
    }

    # Scaling longitude from range [-70, -78] to [0, 1]
    for lon_col in ["pickup_longitude", "dropoff_longitude"]:
        transformed[lon_col] = layers.Lambda(
            lambda x: (x + 78) / 8.0, name=f"scale_{lon_col}"
        )(inputs[lon_col])

    # Scaling latitude from range [37, 45] to [0, 1]
    for lat_col in ["pickup_latitude", "dropoff_latitude"]:
        transformed[lat_col] = layers.Lambda(
            lambda x: (x - 37) / 8.0, name=f"scale_{lat_col}"
        )(inputs[lat_col])

    # Adding Euclidean dist (no need to be accurate: NN will calibrate it)
    transformed["euclidean"] = layers.Lambda(euclidean, name="euclidean")(
        [
            inputs["pickup_longitude"],
            inputs["pickup_latitude"],
            inputs["dropoff_longitude"],
            inputs["dropoff_latitude"],
        ]
    )
    feature_columns["euclidean"] = fc.numeric_column("euclidean")

    # hour of day from timestamp of form '2010-02-08 09:17:00+00:00'
    transformed["hourofday"] = layers.Lambda(
        lambda x: tf.strings.to_number(
            tf.strings.substr(x, 11, 2), out_type=tf.dtypes.int32
        ),
        name="hourofday",
    )(inputs["pickup_datetime"])
    feature_columns["hourofday"] = fc.indicator_column(
        fc.categorical_column_with_identity("hourofday", num_buckets=24)
    )

    latbuckets = np.linspace(0, 1, nbuckets).tolist()
    lonbuckets = np.linspace(0, 1, nbuckets).tolist()
    b_plat = fc.bucketized_column(
        feature_columns["pickup_latitude"], latbuckets
    )
    b_dlat = fc.bucketized_column(
        feature_columns["dropoff_latitude"], latbuckets
    )
    b_plon = fc.bucketized_column(
        feature_columns["pickup_longitude"], lonbuckets
    )
    b_dlon = fc.bucketized_column(
        feature_columns["dropoff_longitude"], lonbuckets
    )
    ploc = fc.crossed_column([b_plat, b_plon], nbuckets * nbuckets)
    dloc = fc.crossed_column([b_dlat, b_dlon], nbuckets * nbuckets)
    pd_pair = fc.crossed_column([ploc, dloc], nbuckets ** 4)
    feature_columns["pickup_and_dropoff"] = fc.embedding_column(pd_pair, 100)

    return transformed, feature_columns


def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))


def build_dnn_model(nbuckets, nnsize, lr):
    STRING_COLS = ["pickup_datetime"]
    NUMERIC_COLS = set(CSV_COLUMNS) - {LABEL_COLUMN, "key"} - set(STRING_COLS)
    inputs = {
        colname: layers.Input(name=colname, shape=(), dtype="float32")
        for colname in NUMERIC_COLS
    }
    inputs.update(
        {
            colname: layers.Input(name=colname, shape=(), dtype="string")
            for colname in STRING_COLS
        }
    )

    # transforms
    transformed, feature_columns = transform(
        inputs, NUMERIC_COLS, STRING_COLS, nbuckets=nbuckets
    )
    dnn_inputs = layers.DenseFeatures(feature_columns.values())(transformed)

    x = dnn_inputs
    for layer, nodes in enumerate(nnsize):
        x = layers.Dense(nodes, activation="relu", name=f"h{layer}")(x)
    output = layers.Dense(1, name="fare")(x)

    model = models.Model(inputs, output)
    lr_optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=lr_optimizer, loss="mse", metrics=[rmse, "mse"])

    return model


def train_and_evaluate(hparams):
    batch_size = hparams["batch_size"]
    nbuckets = hparams["nbuckets"]
    lr = hparams["lr"]
    nnsize = hparams["nnsize"]
    eval_data_path = hparams["eval_data_path"]
    num_evals = hparams["num_evals"]
    num_examples_to_train_on = hparams["num_examples_to_train_on"]
    output_dir = hparams["output_dir"]
    train_data_path = hparams["train_data_path"]

    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    savedmodel_dir = os.path.join(output_dir, "export/savedmodel")
    model_export_path = os.path.join(savedmodel_dir, timestamp)
    checkpoint_path = os.path.join(output_dir, "checkpoints")
    tensorboard_path = os.path.join(output_dir, "tensorboard")

    if tf.io.gfile.exists(output_dir):
        tf.io.gfile.rmtree(output_dir)

    model = build_dnn_model(nbuckets, nnsize, lr)
    logging.info(model.summary())

    trainds = create_train_dataset(train_data_path, batch_size)
    evalds = create_eval_dataset(eval_data_path, batch_size)

    steps_per_epoch = num_examples_to_train_on // (batch_size * num_evals)

    checkpoint_cb = callbacks.ModelCheckpoint(
        checkpoint_path, save_weights_only=True, verbose=1
    )
    tensorboard_cb = callbacks.TensorBoard(tensorboard_path)

    history = model.fit(
        trainds,
        validation_data=evalds,
        epochs=num_evals,
        steps_per_epoch=max(1, steps_per_epoch),
        verbose=2,  # 0=silent, 1=progress bar, 2=one line per epoch
        callbacks=[checkpoint_cb, tensorboard_cb],
    )

    # Exporting the model with default serving function.
    model.save(model_export_path)
    return history

Overwriting ./taxifare/trainer/model.py


### Modify code to read data from and write checkpoint files to GCS 

If you look closely above, you'll notice a new function, `train_and_evaluate` that wraps the code that actually trains the model. This allows us to parametrize the training by passing a dictionary of parameters to this function (e.g, `batch_size`, `num_examples_to_train_on`, `train_data_path` etc.)

This is useful because the output directory, data paths and number of train steps will be different depending on whether we're training locally or in the cloud. Parametrizing allows us to use the same code for both.

We specify these parameters at run time via the command line. Which means we need to add code to parse command line parameters and invoke `train_and_evaluate()` with those params. This is the job of the `task.py` file. 

In [13]:
%%writefile taxifare/trainer/task.py
import argparse

from trainer import model

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--batch_size",
        help="Batch size for training steps",
        type=int,
        default=32,
    )
    parser.add_argument(
        "--eval_data_path",
        help="GCS location pattern of eval files",
        required=True,
    )
    parser.add_argument(
        "--nnsize",
        help="Hidden layer sizes (provide space-separated sizes)",
        nargs="+",
        type=int,
        default=[32, 8],
    )
    parser.add_argument(
        "--nbuckets",
        help="Number of buckets to divide lat and lon with",
        type=int,
        default=10,
    )
    parser.add_argument(
        "--lr", help="learning rate for optimizer", type=float, default=0.001
    )
    parser.add_argument(
        "--num_evals",
        help="Number of times to evaluate model on eval data training.",
        type=int,
        default=5,
    )
    parser.add_argument(
        "--num_examples_to_train_on",
        help="Number of examples to train on.",
        type=int,
        default=100,
    )
    parser.add_argument(
        "--output_dir",
        help="GCS location to write checkpoints and export models",
        required=True,
    )
    parser.add_argument(
        "--train_data_path",
        help="GCS location pattern of train files containing eval URLs",
        required=True,
    )
    parser.add_argument(
        "--job-dir",
        help="this model ignores this field, but it is required by gcloud",
        default="junk",
    )
    args = parser.parse_args()
    hparams = args.__dict__
    hparams.pop("job-dir", None)

    model.train_and_evaluate(hparams)

Overwriting taxifare/trainer/task.py


### Run trainer module package locally

Now we can test our training code locally as follows using the local test data. We'll run a very small training job over a single file with a small batch size and one eval step.

In [14]:
%%bash

EVAL_DATA_PATH=./taxifare/tests/data/taxi-valid*
TRAIN_DATA_PATH=./taxifare/tests/data/taxi-train*
OUTPUT_DIR=./taxifare-model

test ${OUTPUT_DIR} && rm -rf ${OUTPUT_DIR}
export PYTHONPATH=${PYTHONPATH}:${PWD}/taxifare
    
python3 -m trainer.task \
--eval_data_path $EVAL_DATA_PATH \
--output_dir $OUTPUT_DIR \
--train_data_path $TRAIN_DATA_PATH \
--batch_size 5 \
--num_examples_to_train_on 100 \
--num_evals 1 \
--nbuckets 10 \
--lr 0.001 \
--nnsize 32 8

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dropoff_latitude (InputLayer)   [(None,)]            0                                            
__________________________________________________________________________________________________
dropoff_longitude (InputLayer)  [(None,)]            0                                            
__________________________________________________________________________________________________
pickup_longitude (InputLayer)   [(None,)]            0                                            
__________________________________________________________________________________________________
pickup_latitude (InputLayer)    [(None,)]            0                                            
______________________________________________________________________________________________

2021-08-25 21:35:30.945143: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2021-08-25 21:35:31.267602: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-25 21:35:31.267649: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-08-25 21:35:31.267760: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-08-25 21:35:31.628974: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-25 21:35:31.629561: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2199995000 Hz
2021-08-25 21:35:33.044544: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-25 21:35:33.044694: I tensorflow/core/profiler/lib/profiler_sessi

### Run your training package on Cloud AI Platform

Once the code works in standalone mode locally, you can run it on Cloud AI Platform. To submit to the Cloud we use [`gcloud ai-platform jobs submit training [jobname]`](https://cloud.google.com/sdk/gcloud/reference/ml-engine/jobs/submit/training) and simply specify some additional parameters for AI Platform Training Service:
- jobid: A unique identifier for the Cloud job. We usually append system time to ensure uniqueness
- region: Cloud region to train in. See [here](https://cloud.google.com/ml-engine/docs/tensorflow/regions) for supported AI Platform Training Service regions

The arguments before `-- \` are for AI Platform Training Service.
The arguments after `-- \` are sent to our `task.py`.

Because this is on the entire dataset, it will take a while. You can monitor the job from the GCP console in the Cloud AI Platform section.

In [15]:
%%bash

# Output directory and jobID
OUTDIR=gs://${BUCKET}/taxifare/trained_model_$(date -u +%y%m%d_%H%M%S)
JOBID=taxifare_$(date -u +%Y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBID}
gsutil -m rm -rf ${OUTDIR}

# Model and training hyperparameters
BATCH_SIZE=50
NUM_EXAMPLES_TO_TRAIN_ON=100
NUM_EVALS=100
NBUCKETS=10
LR=0.001
NNSIZE="32 8"

# GCS paths
GCS_PROJECT_PATH=gs://$BUCKET/taxifare
DATA_PATH=$GCS_PROJECT_PATH/data
TRAIN_DATA_PATH=$DATA_PATH/taxi-train*
EVAL_DATA_PATH=$DATA_PATH/taxi-valid*

#TODO 2
gcloud ai-platform jobs submit training $JOBID \
    --module-name=trainer.task \
    --package-path=taxifare/trainer \
    --staging-bucket=gs://${BUCKET} \
    --python-version=3.7 \
    --runtime-version=${TFVERSION} \
    --region=${REGION} \
    -- \
    --eval_data_path $EVAL_DATA_PATH \
    --output_dir $OUTDIR \
    --train_data_path $TRAIN_DATA_PATH \
    --batch_size $BATCH_SIZE \
    --num_examples_to_train_on $NUM_EXAMPLES_TO_TRAIN_ON \
    --num_evals $NUM_EVALS \
    --nbuckets $NBUCKETS \
    --lr $LR \
    --nnsize $NNSIZE 

gs://dsparing-sandbox/taxifare/trained_model_210825_213539 us-central1 taxifare_210825_213539
jobId: taxifare_210825_213539
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [taxifare_210825_213539] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe taxifare_210825_213539

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs taxifare_210825_213539


### (Optional) Run your training package using Docker container

AI Platform Training also supports training in custom containers, allowing users to bring their own Docker containers with any pre-installed ML framework or algorithm to run on AI Platform Training. 

In this last section, we'll see how to submit a Cloud training job using a customized Docker image. 

Containerizing our `./taxifare/trainer` package involves 3 steps:

* Writing a Dockerfile in `./taxifare`
* Building the Docker image
* Pushing it to the Google Cloud container registry in our GCP project

The `Dockerfile` specifies
1. How the container needs to be provisioned so that all the dependencies in our code are satisfied
2. Where to copy our trainer Package in the container
3. What command to run when the container is ran (the `ENTRYPOINT` line)

In [16]:
%%writefile ./taxifare/Dockerfile
FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-3:latest
# TODO 3

COPY . /code

WORKDIR /code

ENTRYPOINT ["python3", "-m", "trainer.task"]

Overwriting ./taxifare/Dockerfile


In [17]:
PROJECT_DIR = !cd ./taxifare &&pwd
PROJECT_DIR = PROJECT_DIR[0]
IMAGE_NAME = "taxifare_training_container"
DOCKERFILE = f"{PROJECT_DIR}/Dockerfile"
IMAGE_URI = f"gcr.io/{PROJECT}/{IMAGE_NAME}"

%env PROJECT_DIR=$PROJECT_DIR
%env IMAGE_NAME=$IMAGE_NAME
%env DOCKERFILE=$DOCKERFILE
%env IMAGE_URI=$IMAGE_URI

env: PROJECT_DIR=/home/jupyter/asl-ml-immersion/notebooks/building_production_ml_systems/solutions/taxifare
env: IMAGE_NAME=taxifare_training_container
env: DOCKERFILE=/home/jupyter/asl-ml-immersion/notebooks/building_production_ml_systems/solutions/taxifare/Dockerfile
env: IMAGE_URI=gcr.io/dsparing-sandbox/taxifare_training_container


In [18]:
!docker build $PROJECT_DIR -f $DOCKERFILE -t $IMAGE_URI

Sending build context to Docker daemon  135.7kB
Step 1/4 : FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-3:latest
latest: Pulling from deeplearning-platform-release/tf2-cpu.2-3

53061382: Pulling fs layer 
b3ffe450: Pulling fs layer 
b55a8dd4: Pulling fs layer 
636061ee: Pulling fs layer 
b700ef54: Pulling fs layer 
b8a62d1d: Pulling fs layer 
21c1ca9f: Pulling fs layer 
4d8bd926: Pulling fs layer 
3af94ed9: Pulling fs layer 
7b4ff712: Pulling fs layer 
c4033f73: Pulling fs layer 
7e46914f: Pulling fs layer 
85a6dbe1: Pulling fs layer 
36b1d447: Pulling fs layer 
1ce4acee: Pulling fs layer 
3951fafa: Pulling fs layer 
6d703335: Pulling fs layer 
b2a87d69: Pulling fs layer 
Digest: sha256:f9a39f2155f16573f50a1c7587ccb26e19e23b9b767a7c503ce374ead34c2e83KExtracting    210MB/332.3MB
Status: Downloaded newer image for gcr.io/deeplearning-platform-release/tf2-cpu.2-3:latest
 ---> 06b564b6e868
Step 2/4 : COPY . /code
 ---> abb10b1691b3
Step 3/4 : WORKDIR /code
 ---> Running in eaa4079c9

In [19]:
!docker push $IMAGE_URI

Using default tag: latest
The push refers to repository [gcr.io/dsparing-sandbox/taxifare_training_container]

899878ee: Preparing 
7a187b52: Preparing 
12393cf7: Preparing 
eea30d04: Preparing 
cb08ebd3: Preparing 
43a0b8a6: Preparing 
e5e33f1e: Preparing 
8e8f4996: Preparing 
d3afbbd2: Preparing 
3a0d9161: Preparing 
27f9bc83: Preparing 
839b703f: Preparing 
038361c3: Preparing 
72ab02bc: Preparing 
f59ba4e4: Preparing 
bf18a086: Preparing 
178bb296: Preparing 
82d95108: Preparing 
97c8aec4: Preparing 
97c8aec4: Mounted from deeplearning-platform-release/tf2-cpu.2-3 latest: digest: sha256:efa750b55dd99356d09d9fbbe14087887c3b8b21c07448caf52e3c4386115cb9 size: 4507


**Remark:** If you prefer to build the container image from the command line, we have written a script for that `./taxifare/scripts/build.sh`. This script reads its configuration from the file `./taxifare/scripts/env.sh`. You can configure these arguments the way you want in that file. You can also simply type `make build` from within `./taxifare` to build the image (which will invoke the build script). Similarly, we wrote the script `./taxifare/scripts/push.sh` to push the Docker image, which you can also trigger by typing `make push` from within `./taxifare`.

### Train using a custom container on AI Platform

To submit to the Cloud we use [`gcloud ai-platform jobs submit training [jobname]`](https://cloud.google.com/sdk/gcloud/reference/ml-engine/jobs/submit/training) and simply specify some additional parameters for AI Platform Training Service:
- jobname: A unique identifier for the Cloud job. We usually append system time to ensure uniqueness
- master-image-uri: The uri of the Docker image we pushed in the Google Cloud registry
- region: Cloud region to train in. See [here](https://cloud.google.com/ml-engine/docs/tensorflow/regions) for supported AI Platform Training Service regions


The arguments before `-- \` are for AI Platform Training Service.
The arguments after `-- \` are sent to our `task.py`.

You can track your job and view logs using [cloud console](https://console.cloud.google.com/mlengine/jobs).

In [20]:
%%bash

# Output directory and jobID
OUTDIR=gs://${BUCKET}/taxifare/trained_model
JOBID=taxifare_container_$(date -u +%Y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBID}
gsutil -m rm -rf ${OUTDIR}

# Model and training hyperparameters
BATCH_SIZE=50
NUM_EXAMPLES_TO_TRAIN_ON=100
NUM_EVALS=100
NBUCKETS=10
NNSIZE="32 8"

# AI-Platform machines to use for training
MACHINE_TYPE=n1-standard-4
SCALE_TIER=CUSTOM

# GCS paths.
GCS_PROJECT_PATH=gs://$BUCKET/taxifare
DATA_PATH=$GCS_PROJECT_PATH/data
TRAIN_DATA_PATH=$DATA_PATH/taxi-train*
EVAL_DATA_PATH=$DATA_PATH/taxi-valid*

IMAGE_NAME=taxifare_training_container
IMAGE_URI=gcr.io/$PROJECT/$IMAGE_NAME

gcloud ai-platform jobs submit training $JOBID \
   --staging-bucket=gs://$BUCKET \
   --region=$REGION \
   --master-image-uri=$IMAGE_URI \
   --master-machine-type=$MACHINE_TYPE \
   --scale-tier=$SCALE_TIER \
  -- \
  --eval_data_path $EVAL_DATA_PATH \
  --output_dir $OUTDIR \
  --train_data_path $TRAIN_DATA_PATH \
  --batch_size $BATCH_SIZE \
  --num_examples_to_train_on $NUM_EXAMPLES_TO_TRAIN_ON \
  --num_evals $NUM_EVALS \
  --nbuckets $NBUCKETS \
  --nnsize $NNSIZE 


gs://dsparing-sandbox/taxifare/trained_model us-central1 taxifare_container_210825_213816
jobId: taxifare_container_210825_213816
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [taxifare_container_210825_213816] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe taxifare_container_210825_213816

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs taxifare_container_210825_213816


**Remark:** If you prefer submitting your jobs for training on the AI-platform using the command line, we have written the `./taxifare/scripts/submit.sh` for you (that you can also invoke using `make submit` from within `./taxifare`). As the other scripts, it reads it configuration variables from `./taxifare/scripts/env.sh`.

Copyright 2021 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License